In [1]:
!ls yolov5/data/datasets/coco

LICENSE          images           train2017.cache
README.txt       labels           train2017.txt
annotations      test-dev2017.txt val2017.txt


In [2]:
!ls yolov5/data/datasets/coco/images

train2017 val2017


In [3]:
!ls yolov5/data/datasets/coco/labels

train2017 val2017


In [4]:
!mkdir data/brain_tumor_coco
!mkdir data/brain_tumor_coco/images
!mkdir data/brain_tumor_coco/images/train
!mkdir data/brain_tumor_coco/images/val
!mkdir data/brain_tumor_coco/images/test
!mkdir data/brain_tumor_coco/labels
!mkdir data/brain_tumor_coco/labels/train
!mkdir data/brain_tumor_coco/labels/val

In [5]:
!ls data/brain_tumor_coco/

images labels


In [6]:
!ls data/brain_tumor_coco/images

test  train val


In [7]:
!ls data/brain_tumor_coco/labels

train val


In [8]:
!cp data/brain_tumor_dataset/test/* data/brain_tumor_coco/images/test

In [9]:
# !ls data/brain_tumor_coco/images/test

# 1. Chuẩn bị dữ liệu ảnh

In [10]:
import pandas as pd
from tqdm import tqdm
from PIL import Image

from glob import glob
import json
import os

In [11]:
df = pd.read_csv('data/brain_tumor_dataset/brain_multi.csv')
df = df.sample(frac=1)

In [12]:
train_df = df.iloc[:4000]
train_df

,image_name,label,xmin,ymin,xmax,ymax,confidence
1424,Cancer-588-_jpg.rf.61a7d8d8c55e5af4b9349577f0e...,tumor,199,244,289,339,0.084638
2077,Cancer-1464-_jpg.rf.e710000bad85ff191afa76e9fd...,tumor,73,325,280,507,0.003858
3555,y490_jpg.rf.5efa7da099bbba1b48b2f97c07463c24.jpg,tumor,293,100,500,277,0.937380
945,Cancer-1562-_jpg.rf.732b2587bf2538e020846e32de...,tumor,387,278,521,403,0.891409
3787,Not-Cancer-1150-_jpg.rf.5523b453b163a35182cb9c...,normal,0,0,640,640,0.933751
...,...,...,...,...,...,...,...
5166,y88_jpg.rf.db57d0561b6f4a1ceee9f7fa470beb7a.jpg,tumor,78,294,351,568,0.582480
4695,y432_jpg.rf.2f4d1b8019906ed17fb6d1d94db74328.jpg,tumor,183,343,315,473,0.173577
5832,Not-Cancer-645-_jpg.rf.c8de15f23131375ed57f6cd...,normal,0,0,640,640,0.502748
333,y17_jpg.rf.b32160047f4e3e4d46593be3d6b240bb.jpg,tumor,205,107,463,323,0.411765


In [13]:
val_df = df.iloc[4000:]
val_df

,image_name,label,xmin,ymin,xmax,ymax,confidence
5243,Not-Cancer-840-_jpg.rf.c9459940a7f0926bcb69396...,normal,0,0,640,640,0.621680
3456,Cancer-1878-_jpg.rf.ba9c903d268970a7cc7ef44c35...,tumor,335,134,582,398,0.286610
1547,Not-Cancer-1222-_jpg.rf.7401d967dba8bcd80e9a71...,normal,0,0,640,640,0.610396
713,Cancer-96-_jpg.rf.5b060dd230c62c66f32b3f33b7f1...,tumor,241,291,326,353,0.988793
3356,Not-Cancer-2062-_jpg.rf.4a9083d1eba38d4d81eff1...,normal,0,0,640,640,0.115515
...,...,...,...,...,...,...,...
1210,Cancer-277-_jpg.rf.20a6350d1d2980d7c56e5488361...,tumor,276,262,367,330,0.162630
5571,Not-Cancer-1671-_jpg.rf.ca3dcb2115a4252ee7c925...,normal,0,0,640,640,0.095558
3715,y253_jpg.rf.41db19d22c89e84128d248bd6facb709.jpg,tumor,323,259,518,527,0.253680
4822,Cancer-1642-_jpg.rf.167cfcb624e6e43f3f85617684...,tumor,115,229,255,339,0.635797


In [14]:
for image_name in tqdm(train_df.image_name.to_list()):
    os.system(f'cp data/brain_tumor_dataset/train/{image_name} data/brain_tumor_coco/images/train/')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:09<00:00, 402.56it/s]


In [15]:
for image_name in tqdm(val_df.image_name.to_list()):
    os.system(f'cp data/brain_tumor_dataset/train/{image_name} data/brain_tumor_coco/images/val/')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1870/1870 [00:04<00:00, 401.15it/s]


# 2. Chuẩn bị label

In [16]:
train_df[['image_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax']]

,image_name,label,xmin,ymin,xmax,ymax
1424,Cancer-588-_jpg.rf.61a7d8d8c55e5af4b9349577f0e...,tumor,199,244,289,339
2077,Cancer-1464-_jpg.rf.e710000bad85ff191afa76e9fd...,tumor,73,325,280,507
3555,y490_jpg.rf.5efa7da099bbba1b48b2f97c07463c24.jpg,tumor,293,100,500,277
945,Cancer-1562-_jpg.rf.732b2587bf2538e020846e32de...,tumor,387,278,521,403
3787,Not-Cancer-1150-_jpg.rf.5523b453b163a35182cb9c...,normal,0,0,640,640
...,...,...,...,...,...,...
5166,y88_jpg.rf.db57d0561b6f4a1ceee9f7fa470beb7a.jpg,tumor,78,294,351,568
4695,y432_jpg.rf.2f4d1b8019906ed17fb6d1d94db74328.jpg,tumor,183,343,315,473
5832,Not-Cancer-645-_jpg.rf.c8de15f23131375ed57f6cd...,normal,0,0,640,640
333,y17_jpg.rf.b32160047f4e3e4d46593be3d6b240bb.jpg,tumor,205,107,463,323


In [17]:
with open('yolov5/data/datasets/coco/labels/val2017/000000000139.txt', 'r') as file:
    lines = file.readlines()

# Format: object_index - x_center - y_center - width - height
lines

['58 0.389578 0.416103 0.0385937 0.163146\n',
 '62 0.127641 0.505153 0.233312 0.2227\n',
 '62 0.934195 0.583462 0.127109 0.184812\n',
 '56 0.604656 0.632547 0.0875 0.241385\n',
 '56 0.502508 0.627324 0.0966094 0.231174\n',
 '56 0.669195 0.618991 0.0471406 0.190986\n',
 '56 0.512797 0.528251 0.0337187 0.0272066\n',
 '0 0.686445 0.53196 0.0828906 0.323967\n',
 '0 0.612484 0.446197 0.023625 0.0838967\n',
 '68 0.811859 0.501725 0.0230312 0.0374883\n',
 '72 0.78632 0.536373 0.0317031 0.254249\n',
 '73 0.956156 0.771702 0.0224062 0.1073\n',
 '73 0.96825 0.778075 0.020125 0.109014\n',
 '74 0.710555 0.31 0.0218281 0.0513615\n',
 '75 0.886562 0.831608 0.0573125 0.210493\n',
 '75 0.556945 0.516702 0.0177656 0.0529343\n',
 '56 0.651664 0.528826 0.0150469 0.0293897\n',
 '75 0.388047 0.478415 0.0222188 0.041385\n',
 '75 0.533836 0.487946 0.0152031 0.0392723\n',
 '60 0.599984 0.647148 0.196188 0.208756\n']

In [18]:
def prepare_label_txt(image_name, label, x1, y1, x2, y2, dataset_name):
    img = Image.open(f'data/brain_tumor_dataset/train/{image_name}')
    img_w, img_h = img.size[0], img.size[1]
    # Convert from Absolute bounding box to Relative bounding box
    x = (x1 + x2) / 2 / img_w
    y = (y1 + y2) / 2 / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    image_name = image_name.replace('.jpg', '.txt')
    with open(f'data/brain_tumor_coco/labels/{dataset_name}/{image_name}', 'w') as file:
        if label == 'tumor':
            file.write(f'0 {x} {y} {w} {h}')
        else:
            pass

In [19]:
for _, row in tqdm(train_df.iterrows()):
    prepare_label_txt(
        row.image_name, row.label, row.xmin, row.ymin, row.xmax, row.ymax, 'train'
    )

4000it [00:01, 3161.35it/s]


In [20]:
for _, row in tqdm(val_df.iterrows()):
    prepare_label_txt(
        row.image_name, row.label, row.xmin, row.ymin, row.xmax, row.ymax, 'val'
    )

1870it [00:00, 4277.02it/s]


In [21]:
sample_name = os.listdir('data/brain_tumor_coco/labels/val/')[0]
with open(f'data/brain_tumor_coco/labels/val/{sample_name}', 'r') as file:
    lines = file.readlines()
lines

[]

# 3. Chuẩn bị file txt đường dẫn

In [22]:
def create_txt_path_file(root_path, dataset_name):
    file_path_list = glob(os.path.join(root_path, dataset_name, '*'))
    file_path_list = [path.replace('data/brain_tumor_coco', '.') + '\n' for path in file_path_list]
    with open(f'data/brain_tumor_coco/{dataset_name}.txt', 'w') as file:
        file.writelines(file_path_list)

In [23]:
create_txt_path_file('data/brain_tumor_coco/images/', 'val')

In [24]:
create_txt_path_file('data/brain_tumor_coco/images/', 'train')

In [25]:
create_txt_path_file('data/brain_tumor_coco/images/', 'test')

In [26]:
with open('yolov5/data/datasets/coco/val2017.txt', 'r') as file:
    lines = file.readlines()
lines[:10]

['./images/val2017/000000182611.jpg\n',
 './images/val2017/000000335177.jpg\n',
 './images/val2017/000000278705.jpg\n',
 './images/val2017/000000463618.jpg\n',
 './images/val2017/000000568981.jpg\n',
 './images/val2017/000000092416.jpg\n',
 './images/val2017/000000173830.jpg\n',
 './images/val2017/000000476215.jpg\n',
 './images/val2017/000000479126.jpg\n',
 './images/val2017/000000570664.jpg\n']

In [27]:
with open('data/brain_tumor_coco/val.txt', 'r') as file:
    lines = file.readlines()
lines[:10]

['./images/val/Not-Cancer-205-_jpg.rf.516a86e9b123d5e12000fe32483d2d26.jpg\n',
 './images/val/y407_jpg.rf.1bd00e8baf5d468619b53f41b2a9c186.jpg\n',
 './images/val/y484_jpg.rf.9d57b8fc06a357f5bc6cf3ea0d8daf91.jpg\n',
 './images/val/Cancer-2312-_jpg.rf.f804e6eb31b1031ec4c0f2bde40a501f.jpg\n',
 './images/val/Cancer-2145-_jpg.rf.2b64d345c78f026d6689e60433c9bbe0.jpg\n',
 './images/val/Not-Cancer-1237-_jpg.rf.8acd14a63bf82096736be4d5ea02d893.jpg\n',
 './images/val/y325_jpg.rf.f1632e12b65e77a403024cb818883bb2.jpg\n',
 './images/val/y228_jpg.rf.c0eba23d52c9c8d43df1059b55eacf3c.jpg\n',
 './images/val/Cancer-2192-_jpg.rf.19f5e179f2fd698ffe9daf71bd774857.jpg\n',
 './images/val/Cancer-526-_jpg.rf.4f334b85e25c79129d9f624fd2033e55.jpg\n']